In [1]:
import pandas as pd
import numpy as np
data = pd.read_csv('50_Startups.csv')
print(data.iloc[:2])
x=data.iloc[:,0:-1].values
y=data.iloc[:,-1].values

   R&D Spend  Administration  Marketing Spend       State     Profit
0   165349.2       136897.80        471784.10    New York  192261.83
1   162597.7       151377.59        443898.53  California  191792.06


In [2]:
from sklearn.impute import SimpleImputer
SimpleImputer(missing_values=np.nan,strategy='mean').fit_transform(x[:,:-1])

from sklearn.preprocessing import LabelEncoder,OneHotEncoder
x[:,-1]=LabelEncoder().fit_transform(x[:,-1])
x=OneHotEncoder(categorical_features=[-1]).fit_transform(x).toarray()
x=x[:,1:]
print(x[:2])

from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)
print(len(x_train))
print(len(y_train))

[[0.0000000e+00 1.0000000e+00 1.6534920e+05 1.3689780e+05 4.7178410e+05]
 [0.0000000e+00 0.0000000e+00 1.6259770e+05 1.5137759e+05 4.4389853e+05]]
40
40


D:\anaconda\lib\site-packages\sklearn\preprocessing\_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
D:\anaconda\lib\site-packages\sklearn\preprocessing\_encoders.py:390: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


In [3]:
from sklearn.linear_model import LinearRegression
model=LinearRegression().fit(x_train,y_train)
y_pred= model.predict(x_test)
# print(x_train[:2])
# print(y_pred[:5])
# print(y_test[:5])

In [4]:
#param reduction:
#throw away param with large p-value or keep param with small p-value
#large p-value: large p(x|H), etc, 
#(today sun rises from the west|sun rise from the west)=1,
#nothing can be derived from the case
# sun rise from west today dosenot has any impact on Hypotheis.
#on the other hand, etc,
#P(today sun rises from the east|sun rise from the west)=0
#this can tell us that there is defnitly something wrong about our model.

In [5]:
#model: y=b0*1+b1*x1+b2*x2+...
import statsmodels.formula.api as stats
#在x_train 每项前加一个1
#x_train: 40*5 ,axis=1:加入列
x_train=np.append(arr=np.ones((40,1)),values=x_train,axis=1)
print(x_train[:3])

[[1.0000000e+00 0.0000000e+00 0.0000000e+00 4.4069950e+04 5.1283140e+04
  1.9702942e+05]
 [1.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 1.3542692e+05
  0.0000000e+00]
 [1.0000000e+00 1.0000000e+00 0.0000000e+00 7.3994560e+04 1.2278275e+05
  3.0331926e+05]]


In [28]:
x_current = x_train[:,[0,1,2,3,4,5]]
x_pre=np.array([])


threshold=0.08
feature_list=[0,1,2,3,4,5]
while (x_pre!=x_current) and len(feature_list)>2:
    regressor = stats.OLS(y_train,x_current).fit()
    print(regressor.pvalues)
    idx=np.argmax(regressor.pvalues)
    print('***')
    print(idx)
    if regressor.pvalues[idx]>threshold:
        x_pre=x_current
        del feature_list[idx]
        print(feature_list)
        x_current=x_train[:,feature_list]


[3.99587291e-06 8.87485013e-01 9.34394349e-01 8.37712372e-17
 9.68103063e-01 1.55306551e-01]
***
4
[0, 1, 2, 3, 5]
[8.98973577e-15 8.87895882e-01 9.31208763e-01 2.93117449e-18
 1.37348129e-01]
***
2
[0, 1, 3, 5]
[2.88909418e-17 8.25022996e-01 9.13943614e-19 1.31188984e-01]
***
1
[0, 3, 5]
[5.79982910e-18 2.93887263e-19 1.00223841e-01]
***
2
[0, 3]


D:\anaconda\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: elementwise != comparison failed; this will raise an error in the future.
  import sys


In [29]:
regressor.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.947
Model:                            OLS   Adj. R-squared:                  0.944
Method:                 Least Squares   F-statistic:                     327.6
Date:                Tue, 26 Mar 2019   Prob (F-statistic):           2.93e-24
Time:                        12:52:17   Log-Likelihood:                -421.55
No. Observations:                  40   AIC:                             849.1
Df Residuals:                      37   BIC:                             854.2
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       4.639e+04   2950.782     15.722      0.000    4.04e+04    5.24e+04
x1             0.7977      0.046     17.226      0.000       0.704       0.891
x2             0.0277      0.016      1.686      0.100      -0.006       0.061
==============================================================================
Omnibus:                       13.461   Durbin-Watson:                   1.817
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               17.997
Skew:                          -0.962   Prob(JB):                     0.000124
Kurtosis:                       5.663   Cond. No.                     4.87e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.87e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""